In [1]:
!pip install pytorch-lightning segmentation_models_pytorch tacoreader rasterio albumentations --upgrade

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 55.9 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=5d4259b6179070dfcff12df59761c3a8feeba1db9550c87b573809fc8254c166
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b51

In [2]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from datamodule import CoreDataModule
from model import UNet_CD_Sentinel_2
import config
import metrics
import tacoreader
import torch

In [3]:
# Primeiro Treinamento com as imagens 512x512
tb_logger = TensorBoardLogger(config.DIR_LOG, name=config.NAME_MOBILENET)

dataset_l1c = tacoreader.load(["/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0000.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0001.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0002.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0003.part.taco",
                              "/content/drive/MyDrive/taco_CloudSen12/cloudsen12-l1c.0004.part.taco",
                              ])
df_512 = dataset_l1c[(dataset_l1c["label_type"] == "high") & (dataset_l1c["real_proj_shape"] == 509)]

# Define the datamodule
datamodule = CoreDataModule(
    dataframe=df_512,
    batch_size=config.BATCH_SIZE
)

# Define the model
model = UNet_CD_Sentinel_2(
    encoder_name=config.ENCODER_NAME_MOBILENET,
    classes=config.CLASSES,
    in_channels=config.IN_CHANNELS,
    learning_rate=config.LEARNING_RATE,
)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=config.DIR_ROOT_MOBILENET,
    filename="{epoch}-{train_loss:.2f}-{val_loss:.2f}-trainHigh512",
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

earlystopping_callback = pl.callbacks.EarlyStopping(
    monitor="val_loss", patience=12, mode="min"
)

callbacks = [checkpoint_callback, earlystopping_callback]

# Define the trainer
trainer = pl.Trainer(
    max_epochs=config.EPOCHS,
    log_every_n_steps=1,
    callbacks=callbacks,
    accelerator=config.ACCELERATOR,
    precision="16-mixed",
    logger=tb_logger,
    default_root_dir=config.DIR_ROOT_MOBILENET,
)


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [4]:
torch.set_float32_matmul_precision('medium')
# Start the training
trainer.fit(model=model, datamodule=datamodule)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/drive/MyDrive/Unet_4_bands_l1c/lightning_logs/Unet_mobilenet_v2 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params | Mode 
---------------------------------------------------
0 | model | Unet             | 6.6 M  | train
1 | loss  | CrossEntropyLoss | 0      | train
---------------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.519    Total estimated model params size (MB)
285       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [5]:
# Carregar o melhor modelo diretamente
model = UNet_CD_Sentinel_2.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    encoder_name=config.ENCODER_NAME_MOBILENET,
    classes=config.CLASSES,
    in_channels=config.IN_CHANNELS,
    learning_rate=config.LEARNING_RATE,
)

# run val dataset
val_metrics = trainer.validate(model, datamodule=datamodule, verbose=True)
print(val_metrics)

# run test dataset
test_metrics = trainer.test(model, datamodule=datamodule, verbose=True)
print(test_metrics)

acuracia, iou, f1_score, recall, precision, sensitivity = metrics.calculate_metrics(datamodule.test_dataloader(), model.model)

# Salva o modelo treinado
smp_model = model.model
smp_model.save_pretrained(config.DIR_ROOT_MOBILENET + "/" + config.NAME_MOBILENET)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │    0.2857484817504883     │
│      valid_acuracia       │     0.944932222366333     │
│     valid_dataset_iou     │    0.8015817403793335     │
│      valid_f1_score       │     0.889864444732666     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[{'val_loss': 0.2857484817504883, 'valid_acuracia': 0.944932222366333, 'valid_dataset_iou': 0.8015817403793335, 'valid_f1_score': 0.889864444732666}]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_acuracia       │    0.9444195628166199     │
│     test_dataset_iou      │    0.7999193668365479     │
│       test_f1_score       │    0.8888391256332397     │
│         test_loss         │    0.2939014732837677     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.2939014732837677, 'test_acuracia': 0.9444195628166199, 'test_dataset_iou': 0.7999193668365479, 'test_f1_score': 0.8888391256332397}]


Calculando métricas: 100%|██████████| 31/31 [05:06<00:00,  9.89s/it]


Redução: micro
Acurácia no conjunto de teste: 0.9444
IoU no conjunto de teste: 0.7999
F1 no conjunto de teste: 0.8888
Recall no conjunto de teste: 0.8888
Precisião no conjunto de teste: 0.8888
Sensibilidade no conjunto de teste: 0.8888


Redução: macro
Acurácia no conjunto de teste: 0.9444
IoU no conjunto de teste: 0.7024
F1 no conjunto de teste: 0.8143
Recall no conjunto de teste: 0.8284
Precisião no conjunto de teste: 0.8023
Sensibilidade no conjunto de teste: 0.8284


Redução: micro-imagewise
Acurácia no conjunto de teste: 0.9444
IoU no conjunto de teste: 0.8203
F1 no conjunto de teste: 0.8888
Recall no conjunto de teste: 0.8888
Precisião no conjunto de teste: 0.8888
Sensibilidade no conjunto de teste: 0.8888


Redução: macro-imagewise
Acurácia no conjunto de teste: 0.9444
IoU no conjunto de teste: 0.7208
F1 no conjunto de teste: 0.7794
Recall no conjunto de teste: 0.8367
Precisião no conjunto de teste: 0.8443
Sensibilidade no conjunto de teste: 0.8367


